## Injection benchmark

### Setup

In [1]:
import sys
import subprocess
from antidote import __version__, is_compiled
print(f"""
== Python ==
{sys.version}

== Antidote ==
{__version__} {'(cython)' if is_compiled() else ''}
""")


== Python ==
3.9.1 (default, Dec  7 2020, 22:33:43) 
[GCC 9.3.0]

== Antidote ==
0.12.1 (cython)



In [2]:
cat /proc/cpuinfo | grep 'model name' | head -n 1

model name	: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz


### Results
The key take away from those benchmarks, is to avoid using injection on short functions which are called repeatedly, in a loop typically. In the most common use case of dependency injection, service instantiation, the overhead should be negligible.

It should be noted that in most cases the worst scenario is used, as those functions do nothing. In the real world, pure python functions are a lot slower. So to put the following results into perspective, here is the time needed to decode this simple JSON.

In [3]:
import json
# Rough point of comparison
%timeit json.loads('{"name":"John","age":30,"city":"New York"}')

1.61 µs ± 27.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Function call

Injection overhead is here measured with a function which does nothing.

In [4]:
from antidote import world, Service, inject, Provide

class Service1(Service):
    pass

def f(s1: Service1):
    return s1

@inject
def injected_f(s1: Provide[Service1]):
    return s1

In [5]:
# Reference
s1 = world.get[Service1]() # singleton by default
%timeit f(s1)

51.5 ns ± 1.71 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [6]:
# With injection
assert injected_f() == f(s1)
%timeit injected_f()

186 ns ± 3.27 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [7]:
# With injection when no arguments must be provided
assert injected_f(s1) == f(s1)
%timeit injected_f(s1)

96.7 ns ± 0.652 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


### Function call with multiple injections

In [8]:
from antidote import Provide

class Service2(Service):
    pass

class Service3(Service):
    pass

class Service4(Service):
    pass

def f_multi(s1: Service1, s2: Service2, s3: Service3, s4: Service4):
    return s1, s2, s3, s4

@inject
def injected_f_multi(s1: Provide[Service1],
                     s2: Provide[Service2],
                     s3: Provide[Service3],
                     s4: Provide[Service4]):
    return s1, s2, s3, s4

In [9]:
# Reference
args = (world.get(Service1), world.get(Service2), world.get(Service3), world.get(Service4))
%timeit f_multi(*args)

85.1 ns ± 0.518 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [10]:
# With injection
assert injected_f_multi() == f_multi(*args)
%timeit injected_f_multi()

312 ns ± 6.46 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [11]:
# With injection when no arguments must be provided
assert injected_f_multi(*args) == f_multi(*args)
%timeit injected_f_multi(*args)

117 ns ± 1.13 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


### Method call
Overhead when applied on a method

In [12]:
class Dummy:
    def method(self, s1: Service1):
        return s1
    
    def method_multi(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        return s1, s2, s3, s4
    
    @inject
    def injected_method(self, s1: Provide[Service1]):
        return s1
    
    @inject
    def injected_method_multi(self, 
                s1: Provide[Service1],
                s2: Provide[Service2],
                s3: Provide[Service3],
                s4: Provide[Service4]):
        return s1, s2, s3, s4

dummy = Dummy()

In [13]:
# Reference
%timeit dummy.method(s1)

64.9 ns ± 1.1 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [14]:
# With injection
assert dummy.injected_method() == dummy.method(s1)
%timeit dummy.injected_method()

354 ns ± 5.41 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [15]:
# With injection when no arguments must be provided
assert dummy.injected_method(s1) == dummy.method(s1)
%timeit dummy.injected_method(s1)

253 ns ± 7.19 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Method call with multiple injections

In [16]:
class Dummy2:
    def method_multi(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        return s1, s2, s3, s4
    
    @inject
    def injected_method_multi(self, 
                s1: Provide[Service1],
                s2: Provide[Service2],
                s3: Provide[Service3],
                s4: Provide[Service4]):
        return s1, s2, s3, s4

dummy2 = Dummy2()

In [17]:
# Reference
%timeit dummy2.method_multi(*args)

140 ns ± 3.01 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [18]:
# With injection
assert dummy2.injected_method_multi() == dummy2.method_multi(*args)
%timeit dummy2.injected_method_multi()

502 ns ± 9.81 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [19]:
# With injection when no arguments must be provided
assert dummy2.injected_method_multi(*args) == dummy2.method_multi(*args)
%timeit dummy2.injected_method_multi(*args)

303 ns ± 8.93 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Non singleton

We compare the overhead of creating the whole service each time.

In [20]:
class ServiceX(Service):
    __antidote__ = Service.Conf(singleton=False)

def g(s: ServiceX):
    return s

@inject
def injected_g(s: Provide[ServiceX]):
    return s

In [21]:
# Reference
%timeit g(ServiceX())

118 ns ± 3.65 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [22]:
# With injection
assert isinstance(injected_g(), ServiceX)
assert injected_g() is not injected_g()
%timeit injected_g()

372 ns ± 12.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [23]:
# With injection when no arguments must be provided
%timeit injected_g(ServiceX())

165 ns ± 4.12 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


### Object instantiation
Cost of creating wired objects

In [24]:
class Obj:
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4
        
class InjectedObj:
    @inject
    def __init__(self,
                 s1: Provide[Service1],
                 s2: Provide[Service2],
                 s3: Provide[Service3],
                 s4: Provide[Service4]):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4
    

In [25]:
# Reference
%timeit Obj(*args)

349 ns ± 11.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [26]:
# With injection
%timeit InjectedObj()

708 ns ± 36.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [27]:
# With injection when no arguments must be provided
%timeit InjectedObj(*args)

504 ns ± 25.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Factory

In [28]:
from typing import Annotated
from antidote import factory, From

class ServiceF:
    pass

@factory(singleton=False)
def service_factory() -> ServiceF:
    return ServiceF()

def h(s):
    return s

@inject
def injected_h(s: Annotated[ServiceF, From(service_factory)]):
    return s

In [29]:
# Reference
%timeit h(service_factory())

349 ns ± 17 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [30]:
# With injection
assert isinstance(injected_h(), ServiceF)
assert injected_h() is not injected_h()
%timeit injected_h()

422 ns ± 10.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [31]:
# With injection when no arguments must be provided
%timeit injected_h(service_factory())

406 ns ± 10.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Configuration


In [32]:
from antidote import Constants, const

class Conf(Constants):
    A = const('A')
    B = const('B')

    def get(self, key):
        return key
    
def use_config(a, b):
    return a, b

@inject(dependencies=(Conf.A, Conf.B))
def injected_use_config(a, b):
    return a, b

In [33]:
# Reference
conf = Conf()
%timeit use_config(conf.get('A'), conf.get('B'))

204 ns ± 7.35 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [34]:
# With injection
assert injected_use_config() == use_config(conf.get('A'), conf.get('B'))
%timeit injected_use_config()

242 ns ± 3.54 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [35]:
# With injection when no arguments must be provided
%timeit injected_use_config(conf.get('A'), conf.get('B'))

276 ns ± 15.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Custom Provider

In [36]:
import time
from antidote import world, inject
from antidote.core import Provider, DependencyValue
dep = object()
slow = object()

@world.provider
class SlowProvider(Provider):
    def exists(self, dependency):
        return dependency is slow
    
    def provide(self, dependency, container):
        time.sleep(.01)
        return DependencyValue("sleepy")
            

@world.provider
class CustomProvider(Provider):
    def exists(self, dependency):
        return dependency is dep
    
    def provide(self, dependency, container):
        return DependencyValue("Found it !")


@inject(dependencies=dict(d=dep))
def f_custom(d):
    return d

In [37]:
assert f_custom() == "Found it !"
%timeit f_custom()

1.09 µs ± 6.95 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
